### Tensorflow提供了通过变量名称来创建和获取一个变量的机制。通过这个机制，在不同的函数中可以通过变量的名字来调用变量，而不需要将变量通过参数的形式将变量到处传递

In [ ]:
import tensorflow as tf

In [2]:
# 下面这两个变量定义是等价的
v = tf.get_variable('v', shape=[1], initializer=tf.constant_initializer(1.0))
v = tf.Variable(tf.constant(1.0, shape=[1], name='v'))

![初始化函数](./pic/变量_1.png)

1. tf.get_variable与tf.Variable最大的区别在于指定变量名称的参数。对于tf.Variable函数，变量名称是一个可选的参数。但是对于tf.get_variable函数，变量名是一个必填的参数。
1. tf.Variable是一个类，自带很多属性函数；而 tf.get_variable是一个函数;
2. tf.Variable只能生成独一无二的变量，即如果给出的name已经存在，则会自动修改生成新的变量name;
3. tf.get_variable可以用于生成共享变量。默认情况下，该函数会进行变量名检查，如果有重复则会报错。当在指定变量域中声明可以变量共享时，可以重复使用该变量（例如RNN中的参数共享）。

In [3]:
with tf.variable_scope('scope1',reuse=tf.AUTO_REUSE) as scope1:
    x1 = tf.Variable(tf.ones([1]),name='x1')
    x2 = tf.Variable(tf.zeros([1]),name='x1')
    y1 = tf.get_variable('y1',initializer=1.0)
    y2 = tf.get_variable('y1',initializer=0.0)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        print(x1.name,x1.eval())
        print(x2.name,x2.eval())
        print(y1.name,y1.eval())
        print(y2.name,y2.eval())

scope1/x1:0 [1.]
scope1/x1_1:0 [0.]
scope1/y1:0 1.0
scope1/y1:0 1.0


**在名字为‘foo’的命名空间内创建名字为v的变量。**

In [4]:
with tf.variable_scope('foo'):
    v = tf.get_variable('v', [1], initializer=tf.constant_initializer(1.0))

**因为在命名空间‘foo’中已经创建了名字为v的变量，所以下面的代码将会报错。
ValueError: Variable foo/v already exists, disallowed. 
Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? **

In [5]:
with tf.variable_scope('foo'):
    v = tf.get_variable('v', [1])

ValueError: Variable foo/v already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-4-9560048e281c>", line 3, in <module>
    v = tf.get_variable('v', [1], initializer=tf.constant_initializer(1.0))
  File "/home/g/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/g/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


**在生成上下文管理器时，将参数reuse设置为True，这样tf.get_variable将直接获取已经声明的变量。**

In [6]:
with tf.variable_scope('foo', reuse=True):
    v1 = tf.get_variable('v', [1])
    print(v == v1)
    print(v)
    print(v1)

True
<tf.Variable 'foo/v:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'foo/v:0' shape=(1,) dtype=float32_ref>


**将参数reuse设置为True后，tf.get_variable只能获取已经创建的变量。因为在命名空间bar还没有创建变量v，所以下面代码会报错。ValueError: Variable bar/v does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?**

In [7]:
with tf.variable_scope('bar', reuse=True):
    v1 = tf.get_variable('v', [1])

ValueError: Variable bar/v does not exist, or was not created with tf.get_variable(). Did you mean to set reuse=tf.AUTO_REUSE in VarScope?

**Tensorflow中tf.variable_scope是可以嵌套的**

In [8]:
with tf.variable_scope('root'):
    print(tf.get_variable_scope().reuse)
    
    with tf.variable_scope('foo', reuse=True):
        print(tf.get_variable_scope().reuse)
        with tf.variable_scope('bar'):            # 新建一个嵌套的上下文管理器，但不指定resuse，这时reuse的取值会和外面保持一致。
            print(tf.get_variable_scope().reuse)
    print(tf.get_variable_scope().reuse)

False
True
True
False


**tf.variable_scope函数生成的上下文管理器也会创建一个Tensorflow中的命名空间，在命名空间内创建的变量名称都会带上这个命名空间名作为前缀。所以，tf.variable_scope函数除了可以控制tf.get_variable执行的功能外，这个函数也提供了一个管理变量命名空间的方式。**

In [13]:
a1 = tf.get_variable('a', [1])
print(a1.name)

with tf.variable_scope('foo'):
    a2 = tf.get_variable('a', [1])
    print(a2.name)

with tf.variable_scope('foo'):
    with tf.variable_scope('bar'):
        a3 = tf.get_variable('a', [1])
        print(a3.name)
    a4 = tf.get_variable('a1', [1])
    print(a4.name)
        
with tf.variable_scope('', reuse=True):
    a5 = tf.get_variable('foo/bar/a', [1])
    print(a5 == a3)
    a6 = tf.get_variable('foo/a1', [1])
    print(a6 == a4)

a:0
foo/a:0
foo/bar/a:0
foo/a1:0
True
True


**variable_scope与get_variable搭配使用可以使得共享变量。
name_scope主要用来tensorboard可视化**